<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

# Analyzing Wet Snow Extent from SWS Time Series Based on Altitude

## Environment Setup
Before we begin, we need to prepare our environment by installing and importing the necessary R packages.

### Load Required Libraries

In [ ]:
library(terra)
library(fs)
library(ggplot2)

library(magrittr)
library(dplyr)
library(scales)

## Functions

#### Function to Open and Read GeoTIFF Files, Assuming Downloaded Products Have Already Been Extracted

In [ ]:
read_tif <- function(file_tif) {
  ds <- rast(file_tif)
  
  proj <- crs(ds)
  width <- ncol(ds)
  height <- nrow(ds)
  count <- nlyr(ds)
  meta <- list(
    min = minmax(ds)[1, ],
    max = minmax(ds)[2, ],
    mean = global(ds, fun = mean, na.rm = TRUE),
    sd = global(ds, fun = sd, na.rm = TRUE)
  )
  
  list(data = ds, width = width, height = height, count = count, meta = meta, proj = proj)
}

## Usage

#### Downloading SWS and DEM Data

To begin our analysis, we first need to download the necessary Snow Water Equivalent (SWS) and Digital Elevation Model (DEM) data. This can be accomplished using the HDA (Harmonized Data Access) client.  
The SWS data provides information on the extent of wet snow, while the DEM data offers detailed elevation information for the study area.  
By querying the HDA client with specific parameters such as dataset ID, observation period, and bounding box coordinates, we can retrieve and download the relevant datasets to our local directory for further processing and analysis.



In [ ]:
hda_client <- hdar::Client$new()

download_dir <- "../../data/download/snow-and-ice/products"
if (!fs::dir_exists(download_dir)) {
  fs::dir_create(download_dir)
}

query_sws <- jsonlite::toJSON(list(
  "dataset_id" = "EO:CRYO:DAT:HRSI:SWS",
  "startdate" = "2022-11-01T01:00:00Z",
  "enddate" = "2022-12-01T01:00:00Z",
  "bbox" = c(
      6.213305360358289,
      44.881040312330924,
      6.27014257275879,
      44.925714461959465
  )
), auto_unbox = TRUE)

query_dem <- jsonlite::toJSON(list(
  "dataset_id" = "EO:DEM:DAT:COP-DEM_GLO-30-DTED__2023_1",
  "bbox" = c(
      6.213305360358289,
      44.881040312330924,
      6.27014257275879,
      44.925714461959465
  )
), auto_unbox = TRUE)

tryCatch({
    matches <- hda_client$search(query_sws, limit = 3)
    matches$download(download_dir)

    matches <- hda_client$search(query_dem, limit = 1)
    matches$download(download_dir) 
}, error = function(e) {
    print(e)
})

#### Organizing Downloaded Files for Analysis

To facilitate the analysis, we extract the downloaded SWS and DEM files into dedicated directories. This organization ensures that all relevant data is readily accessible and systematically arranged, enhancing the efficiency of subsequent processing and analysis steps.

In [ ]:
prod_dir <- "../../data/processing/snow-and-ice/products"
if (!fs::dir_exists(prod_dir)) {
  fs::dir_create(prod_dir)
}

# Find all zip files
zip_files <- fs::dir_ls(download_dir, regexp = "SWS_.*\\.zip$")

# Create SWS directory if it doesn't exist
sws_dir <- fs::path(prod_dir, "SWS")
if (!fs::dir_exists(sws_dir)) {
    fs::dir_create(sws_dir)
}

# Unzip each file
for (zip_file in zip_files) {
    zip::unzip(zip_file, exdir = sws_dir)
}

In [ ]:
# Find all tar files
tar_files <- fs::dir_ls(download_dir, glob = "*.tar")

dem_dir <- fs::path(prod_dir, "DEM")
if (!fs::dir_exists(dem_dir)) {
    fs::dir_create(dem_dir)
}

# Extract each file
for (tar_file in tar_files) {
    untar(tar_file, exdir = dem_dir)
}

#### Converting DEM from 30m to 60m Resolution

In our analysis, we require a Digital Elevation Model (DEM) with a 60m resolution. However, a 60m DEM is not available for direct download from any source. The Copernicus DEM, which we are using, is available at a 30m resolution. To meet our requirements, we need to convert the 30m DEM to a 60m resolution.

In [ ]:
convert_and_resample <- function(input_dt2, output_tiff, resampled_tiff, x_res_deg, y_res_deg) {

    dt2_dataset <- rast(input_dt2)
    if (is.null(dt2_dataset)) {
        stop(paste("Could not open", input_dt2))
    }

    # Convert .dt2 to GeoTIFF
    writeRaster(dt2_dataset, output_tiff, overwrite = TRUE, filetype = "GTiff")

    tiff_dataset <- rast(output_tiff)
    if (is.null(tiff_dataset)) {
        stop(paste("Could not open", output_tiff))
    }

    # Create an empty raster with the desired resolution
    target_res <- c(x_res_deg, y_res_deg)
    template <- rast(tiff_dataset)
    res(template) <- target_res

    # Resample the GeoTIFF to the desired resolution
    resampled_dataset <- resample(tiff_dataset, template, method = "bilinear")

    if (is.null(resampled_dataset)) {
        stop(paste("Could not resample", output_tiff, "to", resampled_tiff))
    }

    writeRaster(resampled_dataset, resampled_tiff, overwrite = TRUE, filetype = "GTiff")

    print(paste("Successfully converted", input_dt2, "to", resampled_tiff, "with", x_res_deg, "x", y_res_deg, "degrees resolution."))
}

dt2_files <- fs::dir_ls(dem_dir, recurse = TRUE, regexp = ".*\\.dt2$")

resampled_dir <- fs::path(dem_dir, "resampled")
if (!fs::dir_exists(resampled_dir)) {
    fs::dir_create(resampled_dir)
}

# 60 meters to degrees conversion
x_res_deg <- 60 / 111320
y_res_deg <- 60 / 111320

# Convert and resample .dt2 files
for (dt2_file in dt2_files) {
    base_name <- path_ext_remove(path_file(dt2_file))
    output_tiff <- fs::path(resampled_dir, paste0(base_name, ".tif"))
    resampled_tiff <- fs::path(resampled_dir, paste0(base_name, "_60m.tif"))
    convert_and_resample(dt2_file, output_tiff, resampled_tiff, x_res_deg, y_res_deg)
}

In [ ]:
files_wsm <- fs::dir_ls(fs::path(prod_dir, "SWS"), recurse = TRUE, regexp = "SWS_.*_WSM\\.tif")
file_elev <- fs::dir_ls(fs::path(prod_dir, "DEM/resampled"), recurse = TRUE, regexp =  ".*DEM_60m\\.tif")

First, open and read all Wet Snow products for the mountains using the function defined below:

In [ ]:
wsm_datas <- list()

for (file_wsm in files_wsm) {
  wsm_datas <- append(wsm_datas, list(read_tif(file_wsm)$data))
}

Next, open and read the elevation data:

In [ ]:
elev_data <- read_tif(file_elev[1])$data
elev_data[is.na(elev_data)] <- 0

After loading the data, plot the elevation data and its associated colorbar for visualization.

In [ ]:
nrows <- nrow(elev_data)
ncols <- ncol(elev_data)

elev_df  <- data.frame(
  x = rep(1:nrows, each = nrows), 
  y = rep(1:ncols, times = ncols),
  value = as.vector(elev_data[])
)

options(repr.plot.width = 10, repr.plot.height = 8)
ggplot(elev_df, aes(x = x, y = y, fill = value)) +
  geom_raster() + 
  coord_fixed() +
  scale_fill_viridis_c(breaks = seq(0, 4000, by = 500)) +
  scale_x_continuous(breaks = seq(0, max(elev_df$x), by = 250)) +
  scale_y_continuous(breaks = seq(0, max(elev_df$y), by = 250)) +
  theme_minimal() +
  theme(
      legend.position = "right",
      legend.direction = "vertical",
      legend.justification = "center",
      legend.text = element_text(size = 16),
      legend.title = element_blank(),
      legend.key.height = unit(2, "cm"),
      axis.text = element_text(size = 14),
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
  ) +
  labs(fill = "Elevation")

Then, plot the first SWS product of the time series, ensuring to add a standard colorbar (not the one attached to the product)

For details on the SWS product coding, please refer to the Product User Manual (https://land.copernicus.eu/user-corner/technical-library/hrsi-snow-pum):

In [ ]:
wsm_data <- wsm_datas[[2]]
wsm_data[is.na(wsm_data)] <- 0

nrows <- nrow(wsm_data)
ncols <- ncol(wsm_data)

wsm_df  <- data.frame(
  x = rep(1:nrows, each = nrows), 
  y = rep(1:ncols, times = ncols),
  value = as.vector(wsm_data[])
)

options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(wsm_df, aes(x = x, y = y, fill = value)) +
  geom_raster() + 
  scale_fill_viridis_c(breaks = seq(0, 250, by = 20)) +
  coord_fixed() +
  scale_x_continuous(breaks = seq(0, max(elev_df$x), by = 250)) +
  scale_y_continuous(breaks = seq(0, max(elev_df$y), by = 250)) +
  theme_minimal() +
  theme(
      legend.position = "right",
      legend.direction = "vertical",
      legend.justification = "center",
      legend.text = element_text(size = 16),
      legend.title = element_blank(),
      legend.key.height = unit(2, "cm"),
      axis.text = element_text(size = 14),
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
  ) +
  labs(fill = "Elevation")


Now, define the elevation intervals to be used for the analysis.
<table><tr><td>
Note: in this example, the elevation range covers 1000 - 4000 m a.s.l., using an altitude interval of 200 m.
</td></tr></table>

In [ ]:
dbin <- 200
bins <- c(seq(0, 1400, by = dbin), Inf)
elevs <- bins[1:(length(bins)-1)] + dbin / 2

Extract the elevation information for wet snow pixels and add this data to a database for all SWS products in the time series.

In [ ]:
val_wet_snow <- 110

hists <- list()
for (wsm_data in wsm_datas) {
  wsm_data[is.na(wsm_data)] <- 0
  sel <- (values(wsm_data) == val_wet_snow)
  
  if (nrow(sel) < nrow(values(elev_data))) {
    sel <- rbind(sel, matrix(FALSE, nrow(values(elev_data)) - nrow(sel), ncol(sel)))
  }
  elev_sel <- values(elev_data)[sel]
  
  hists[[length(hists) + 1]] <- hist(elev_sel, breaks = bins, plot = FALSE)$counts
}

Retrieve the dates from the SWS product file names to be used as legends in the next step:

In [ ]:
labels_wsm <- sapply(files_wsm, function(ele) {
  tools::file_path_sans_ext(basename(ele))
})

Finally, prepare a plot showing the wet snow extent in relation to the altitude for the full SWS time series used as input.

Ensure that the legend (from the previous step) and axis labels are added for clarity.

In [ ]:
# Prepare the data for plotting
plot_data <- data.frame()
for (i in seq_along(hists)) {
  plot_data <- rbind(plot_data, data.frame(
    hist = hists[[i]],
    elevs = elevs,
    label_wsm = substr(labels_wsm[i], 5, 12)
  ))
}

# Plot the data
options(repr.plot.width = 10, repr.plot.height = 8)
ggplot(plot_data, aes(x = hist, y = elevs, color = label_wsm)) +
  geom_line() +
  labs(
    title = "",
    x = "Number of pixels",
    y = "Altitude [m a.s.l.]",
    color = "WSM Label"
  ) +
  scale_x_continuous(breaks = seq(0, 20000, by = 2000), limits = c(0, 20000)) +
  scale_y_continuous(breaks = seq(0, max(plot_data$elevs), by = 200)) +
  theme_minimal() +
  theme(
    legend.position = "right",
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 16)
  )


***
><span style = "font-family:Verdana; font-size:0.7em">Copyright © <font color='darkblue'>2022</font>, by ENVEO IT GmbH.</span>  
<span style = "font-family:Verdana; font-size:0.7em">Contributors: Lars Keuris,  Gabriele Schwaizer</span>  
<span style = "font-family:Verdana; font-size:0.7em">URL: www.enveo.at</span> 
***
<p style = "font-family:Verdana; font-size:0.7em; line-height:0.5">Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files, to use the Software without restriction.</p>  
<p style = "font-family:Verdana; font-size:0.7em; line-height:1.15; text-align:justify">THE SOFTWARE IS PROVIDED AS IS, WITHOUT WARRANTY OF ANYKIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. <b>IN NO EVENT SHALL THE COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DIRECT INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</b></p>

*** 